# Gaussian Process Regression (GPR) with UQ — Tuned (skopt)

This notebook performs Gaussian Process Regression on the uploaded
`concrete_data.csv` dataset (subsampled to 400 points for speed) and
performs hyperparameter tuning using scikit-optimize (`skopt`) with
two objectives: CV-RMSE (BayesSearchCV) and negative log marginal likelihood (gp_minimize).

Run **Kernel -> Restart & Run All** in Jupyter or **Runtime -> Run all** in Colab.

## Requirements

Install required packages if missing:

```
pip install numpy pandas scikit-learn matplotlib seaborn scikit-optimize scipy
```

In [ ]:

# Load dataset (uploaded by you)
import pandas as pd, numpy as np
file_path = "/mnt/data/concrete_data.csv"
df = pd.read_csv(file_path)
print("Loaded:", file_path, "shape:", df.shape)
df.head()


In [ ]:

# Subsample to at most 400 samples
max_samples = 400
if len(df) > max_samples:
    df = df.sample(n=max_samples, random_state=42).reset_index(drop=True)
    print("Subsampled to", len(df))
else:
    print("Using full dataset of", len(df))


In [ ]:

# Preprocess: train/test split and scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
print("Train/test shapes:", X_train_s.shape, X_test_s.shape)


In [ ]:

# Baseline GPR (internal LML tuning)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=1.5) + WhiteKernel(noise_level=1.0)
gpr_base = GaussianProcessRegressor(kernel=kernel, normalize_y=True, n_restarts_optimizer=2, random_state=0)
print('Fitting baseline GPR...')
gpr_base.fit(X_train_s, y_train)
y_pred_base, y_std_base = gpr_base.predict(X_test_s, return_std=True)
print('Baseline kernel:', gpr_base.kernel_)
print('Baseline RMSE:', sqrt(mean_squared_error(y_test, y_pred_base)))


In [ ]:

# CV-based tuning via BayesSearchCV (optimize CV RMSE)
try:
    from skopt import BayesSearchCV
    from skopt.space import Real, Integer, Categorical
    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
    from math import sqrt
    from sklearn.metrics import make_scorer, mean_squared_error
    
    print('Running BayesSearchCV to optimize CV RMSE...')
    # Define a param space in terms of kernel objects for coarse search
    def kernel_from_params(ls, sv, nl):
        return ConstantKernel(sv) * RBF(length_scale=ls) + WhiteKernel(noise_level=nl)
    
    # We'll search continuous parameters by encoding them and constructing kernel in the estimator via a custom wrapper below.
    # For simplicity here, we use BayesSearchCV over kernel objects (coarse) and also provide an example of continuous search via gp_minimize later.
    k1 = kernel_from_params(1.0, 1.0, 1e-2)
    k2 = kernel_from_params(0.1, 2.0, 1e-3)
    k3 = kernel_from_params(5.0, 0.5, 1e-1)
    gpr_for_search = GaussianProcessRegressor(normalize_y=True, n_restarts_optimizer=0)
    search_spaces = {'kernel': Categorical([k1, k2, k3])}
    bayes_cv = BayesSearchCV(gpr_for_search, search_spaces, scoring='neg_root_mean_squared_error', n_iter=12, cv=4, random_state=0, n_jobs=1)
    bayes_cv.fit(X_train_s, y_train)
    print('BayesSearchCV best score:', bayes_cv.best_score_)
    print('Best kernel (CV-RMSE):', bayes_cv.best_estimator_.kernel_)
    gpr_cv = bayes_cv.best_estimator_
    y_pred_cv, y_std_cv = gpr_cv.predict(X_test_s, return_std=True)
    print('Test RMSE (CV-optimized):', sqrt(mean_squared_error(y_test, y_pred_cv)))
except Exception as e:
    print('BayesSearchCV failed or skopt not installed:', e)
    gpr_cv = gpr_base
    y_pred_cv, y_std_cv = y_pred_base, y_std_base


In [ ]:

# NLL tuning via gp_minimize: directly minimize negative log marginal likelihood on training data
try:
    from skopt import gp_minimize
    from skopt.space import Real
    from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
    from math import sqrt
    from sklearn.metrics import mean_squared_error
    
    print('Running gp_minimize for NLL optimization (this will fit many GPRs)...')
    space = [Real(1e-2, 10.0, name='length_scale'), Real(1e-2, 10.0, name='signal_var'), Real(1e-6, 1e-1, name='noise_level')]
    
    def objective(params):
        ls, sv, nl = params
        kernel = ConstantKernel(sv) * RBF(length_scale=ls) + WhiteKernel(noise_level=nl)
        gpr = GaussianProcessRegressor(kernel=kernel, normalize_y=True, optimizer=None)  # no internal optimizer for fairness
        gpr.fit(X_train_s, y_train)
        return -gpr.log_marginal_likelihood_value_
    
    res = gp_minimize(objective, space, n_calls=30, random_state=0, n_initial_points=8)
    best_ls, best_sv, best_nl = res.x
    print('Best params (NLL):', res.x, 'best NLL:', res.fun)
    best_kernel = ConstantKernel(best_sv) * RBF(length_scale=best_ls) + WhiteKernel(noise_level=best_nl)
    gpr_nll = GaussianProcessRegressor(kernel=best_kernel, normalize_y=True, n_restarts_optimizer=0, random_state=0)
    gpr_nll.fit(X_train_s, y_train)
    y_pred_nll, y_std_nll = gpr_nll.predict(X_test_s, return_std=True)
    print('Test RMSE (NLL-optimized):', sqrt(mean_squared_error(y_test, y_pred_nll)))
except Exception as e:
    print('gp_minimize failed or skopt not installed:', e)
    gpr_nll = gpr_base
    y_pred_nll, y_std_nll = y_pred_base, y_std_base


In [ ]:

# Compare baseline, CV-optimized, and NLL-optimized models
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error
def mean_nll(y_true, y_pred, y_std):
    var = y_std**2 + 1e-12
    nll = 0.5 * ((y_true - y_pred)**2 / var + np.log(2*np.pi*var))
    return float(np.mean(nll))

def summarise(name, yp, ys):
    rmse = sqrt(mean_squared_error(y_test, yp))
    mae = mean_absolute_error(y_test, yp)
    nll = mean_nll(y_test, yp, ys)
    print(f'{name}: RMSE={rmse:.4f}, MAE={mae:.4f}, mean NLL={nll:.4f}')

print('Model comparison:')
summarise('Baseline', y_pred_base, y_std_base)
summarise('CV-optimized', y_pred_cv, y_std_cv)
summarise('NLL-optimized', y_pred_nll, y_std_nll)


In [ ]:

# Visualization: parity, uncertainty vs error, std distribution (for NLL-optimized model)
import matplotlib.pyplot as plt, seaborn as sns, numpy as np
sns.set(style='whitegrid')
# Parity plot
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_nll, alpha=0.6)
m = min(min(y_test), min(y_pred_nll)); M = max(max(y_test), max(y_pred_nll))
plt.plot([m,M],[m,M],'r--'); plt.xlabel('True'); plt.ylabel('Predicted'); plt.title('Parity: NLL-optimized')
plt.tight_layout(); plt.show()

# Uncertainty vs absolute error
err = np.abs(y_test - y_pred_nll)
plt.figure(figsize=(6,4))
plt.scatter(y_std_nll, err, alpha=0.7)
plt.xlabel('Predictive std'); plt.ylabel('Absolute error'); plt.title('Uncertainty vs Absolute Error')
plt.tight_layout(); plt.show()

# Std distribution
plt.figure(figsize=(6,4))
sns.histplot(y_std_nll, bins=30, kde=True)
plt.xlabel('Predictive std'); plt.title('Predictive std distribution (NLL model)'); plt.tight_layout(); plt.show()


In [ ]:

# Save kernels/results
results = {
    'baseline_kernel': str(gpr_base.kernel_),
    'cv_kernel': str(getattr(gpr_cv, 'kernel_', 'n/a')),
    'nll_kernel': str(getattr(gpr_nll, 'kernel_', 'n/a')),
}
import json
with open('/mnt/data/gpr_tuning_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print('Saved /mnt/data/gpr_tuning_results.json')

## Notes
- If `skopt` is not installed, run `pip install scikit-optimize` then rerun the notebook.
- `gp_minimize` runs multiple GPR fits and can be slow; reduce `n_calls` to speed up.